In [1]:
import pandas as pd
import numpy as np
import sys

In [3]:
sys.path.append('..')

from io import StringIO
from load import load_station_data

In [4]:
# NOAA dataset
# selected 3 regions with 3 stations each - 
# - specifically with different current levels of bleaching alerts - (HAS TO BE CHANGED)
urls = [
    # Great Barrier Reef Region
    "https://coralreefwatch.noaa.gov/product/vs/data/gbr_far_northern.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/torres_strait.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/gbr_northern.txt",
    
    # Polynesia Region
    "https://coralreefwatch.noaa.gov/product/vs/data/samoas.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/northern_cook_islands.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/hawaii.txt",
    
    # Caribbean Region
    "https://coralreefwatch.noaa.gov/product/vs/data/nicaragua.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/panama_atlantic_east.txt",
    "https://coralreefwatch.noaa.gov/product/vs/data/jamaica.txt"

]

In [7]:
# load all stations and combine into one dataset
print("Loading data from all stations")
all_stations_data = []

for url in urls:
    try:
        station_df = load_station_data(url)
        station_name = station_df['Station'].iloc[0]
        print(f"Loaded {station_name} with {len(station_df)} records")
        all_stations_data.append(station_df)
    except Exception as e:
        print(f"Error loading {url}: {str(e)}")


if all_stations_data:
    combined_df = pd.concat(all_stations_data, ignore_index=True)
    
    # create date column 
    if all(col in combined_df.columns for col in ['YYYY', 'MM', 'DD']):
        combined_df['YYYY'] = combined_df['YYYY'].astype(str)
        combined_df['MM'] = combined_df['MM'].astype(str).str.zfill(2)
        combined_df['DD'] = combined_df['DD'].astype(str).str.zfill(2)
        
        combined_df['Date'] = pd.to_datetime(
            combined_df['YYYY'] + '-' + combined_df['MM'] + '-' + combined_df['DD'],
            errors='coerce'
        )
    
    numeric_cols = ['SST_MIN', 'SST_MAX', 'SST@90th_HS', 'SSTA@90th_HS', 
                    '90th_HS>0', 'DHW_from_90th_HS>1', 'BAA_7day_max']
    
    existing_numeric_cols = [col for col in numeric_cols if col in combined_df.columns]
    
    for col in existing_numeric_cols:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    
    # add season column based on the month
    if 'MM' in combined_df.columns:
        combined_df['Season'] = combined_df['MM'].apply(lambda x: 
            'Winter' if x in ['12', '01', '02'] else
            'Spring' if x in ['03', '04', '05'] else
            'Summer' if x in ['06', '07', '08'] else
            'Fall')
    
    # combined dataset shape
    print(f"\nFinal combined dataset shape: {combined_df.shape}")
    print("\nColumns in combined dataset:")
    print(combined_df.columns.tolist())
    
    print("\nFirst few rows of combined data:")
    print(combined_df.head())
    
    # save
    combined_df.to_csv('../../data/noaa/pulled_data/coral_reef_data_combined.csv', index=False)
    print("\nCombined dataset saved to 'coral_reef_data_combined.csv'")
else:
    print("no stations loaded.")

Loading data from all stations
Loaded Far Northern GBR with 14751 records
Loaded Torres Strait with 14749 records
Loaded Northern GBR with 14751 records
Loaded Samoas with 14750 records
Loaded Northern Cook Islands with 14751 records
Loaded Main Hawaiian Islands with 14751 records
Loaded Nicaragua with 14751 records
Loaded Panama Atlantic East with 14751 records
Loaded Jamaica with 14751 records

Final combined dataset shape: (132756, 16)

Columns in combined dataset:
['YYYY', 'MM', 'DD', 'SST_MIN', 'SST_MAX', 'SST@90th_HS', 'SSTA@90th_HS', '90th_HS>0', 'DHW_from_90th_HS>1', 'BAA_7day_max', 'Station', 'Region', 'Latitude', 'Longitude', 'Date', 'Season']

First few rows of combined data:
   YYYY  MM  DD  SST_MIN  SST_MAX  SST@90th_HS  SSTA@90th_HS  90th_HS>0  \
0  1985  01  01    28.82    29.25        29.03        0.7797       0.56   
1  1985  01  02    28.80    29.33        29.02        0.7135       0.55   
2  1985  01  03    28.80    29.27        29.03        0.7219       0.49   
3  1

# GCBD dataset

In [20]:
gcbd_df = pd.read_csv('../../data/gcbd/global_bleaching_environmental.csv', low_memory=False)
noaa_df = pd.read_csv('../../data/noaa/pulled_data/coral_reef_data_combined.csv', low_memory=False)

In [ ]:
latest_date_noaa = pd.to_datetime(noaa_df['Date']).max()
print(f"Latest date in NOAA dataset: {latest_date_noaa}")

oldest_date_noaa = pd.to_datetime(noaa_df['Date']).min()
print(f"Oldest date in NOAA dataset: {oldest_date_noaa}")

Latest date in NOAA dataset: 2025-05-21 00:00:00
Oldest date in NOAA dataset: 1985-01-01 00:00:00


In [32]:
latest_date_gcbd = pd.to_datetime(gcbd_df['Date']).max()
print(f"Latest date in GCBD dataset: {latest_date_gcbd}")

latest_date_gcbd = pd.to_datetime(gcbd_df['Date']).min()
print(f"Latest date in GCBD dataset: {latest_date_gcbd}")

Latest date in GCBD dataset: 2020-08-15 00:00:00
Latest date in GCBD dataset: 1980-06-15 00:00:00


In [ ]:
bco_dates = set(pd.to_datetime(gcbd_df['Date']).unique())
noaa_dates = set(pd.to_datetime(noaa_df['Date']).unique())

overlap = len(bco_dates & noaa_dates)  # intersection
total_dates = len(bco_dates | noaa_dates)  # union

print(f"\nNumber of unique dates:\nGCBD: {len(bco_dates):,}\nNOAA: {len(noaa_dates):,}")
print(f"Total unique dates across both datasets: {total_dates:,}")
print(f"Number of overlapping dates: {overlap:,}")
print(f"Overlap percentage: {(overlap/total_dates) * 100:.2f}%")



Number of unique dates:
GCBD: 5,212
NOAA: 14,751
Total unique dates across both datasets: 14,760
Number of overlapping dates: 5,203
Overlap percentage: 35.25%


In [13]:
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

Total rows: 41,361
Total columns: 62


In [14]:
# normalizing empty columns
df.replace({'nd': np.nan, 'ND': np.nan, '': np.nan}, inplace=True)


In [123]:
null_pct = df.isna().mean() * 100
to_drop  = null_pct[null_pct > 90].index.tolist()              # >90% missing
const    = df.nunique(dropna=False).loc[lambda x: x <= 1].index # 0 or 1 unique value
drop_all = list(set(to_drop) | set(const))
df.drop(columns=drop_all, inplace=True)
print(f"Dropped {len(drop_all)} cols → {df.shape[1]} left")

Dropped 3 cols → 59 left


In [124]:
print(df.head())

   Site_ID  Sample_ID Data_Source  Latitude_Degrees  Longitude_Degrees  \
0     2501   10324336      Donner            23.163           -82.5260   
1     3467   10324754      Donner           -17.575          -149.7833   
2     1794   10323866      Donner            18.369           -64.5640   
3     8647   10328028      Donner            17.760           -64.5680   
4     8648   10328029      Donner            17.769           -64.5830   

  Ocean_Name Reef_ID            Realm_Name  \
0   Atlantic     NaN     Tropical Atlantic   
1    Pacific     NaN  Eastern Indo-Pacific   
2   Atlantic     NaN     Tropical Atlantic   
3   Atlantic     NaN     Tropical Atlantic   
4   Atlantic     NaN     Tropical Atlantic   

                               Ecoregion_Name      Country_Name  ... TSA_Mean  \
0                     Cuba and Cayman Islands              Cuba  ...    -2.17   
1            Society Islands French Polynesia  French Polynesia  ...    -1.26   
2  Hispaniola Puerto Rico and Lesse

In [125]:
# looking for bleaching related columns - we don't need all of these though, so we could just pick the columns we want to keep
print(f"\n bleaching related columns")
key_terms = ['bleach', 'temperature', 'sst', 'dhw', 'stress', 'severity']
for col in df.columns:
    if any(term in col.lower() for term in key_terms):
        print(f"{col}")
        if df[col].dtype in ['int64', 'float64']:
            print(f"  Range: {df[col].min()} to {df[col].max()}")
        else:
            print(f"  Unique values: {df[col].nunique()}")
        print()


 bleaching related columns
Bleaching_Level
  Unique values: 1

Percent_Bleaching
  Unique values: 2271

ClimSST
  Unique values: 983

Temperature_Kelvin
  Unique values: 1242

Temperature_Mean
  Unique values: 813

Temperature_Minimum
  Unique values: 1023

Temperature_Maximum
  Unique values: 706

Temperature_Kelvin_Standard_Deviation
  Unique values: 398

SSTA
  Unique values: 666

SSTA_Standard_Deviation
  Unique values: 125

SSTA_Mean
  Unique values: 1

SSTA_Minimum
  Unique values: 397

SSTA_Maximum
  Unique values: 584

SSTA_Frequency
  Unique values: 414

SSTA_Frequency_Standard_Deviation
  Unique values: 680

SSTA_FrequencyMax
  Unique values: 321

SSTA_FrequencyMean
  Unique values: 153

SSTA_DHW
  Unique values: 1738

SSTA_DHW_Standard_Deviation
  Unique values: 591

SSTA_DHWMax
  Unique values: 2058

SSTA_DHWMean
  Unique values: 519

TSA_DHW
  Unique values: 1191

TSA_DHW_Standard_Deviation
  Unique values: 416

TSA_DHWMax
  Unique values: 1705

TSA_DHWMean
  Unique value

In [76]:
df['Percent_Bleaching'] = pd.to_numeric(df['Percent_Bleaching'], errors='coerce')
print("\nPercent_Bleaching summary")
print(df['Percent_Bleaching'].describe().round(2))
zeros = (df['Percent_Bleaching'] == 0).sum()
pos   = (df['Percent_Bleaching'] > 0).sum()
miss  = df['Percent_Bleaching'].isna().sum()
print(f"  zeros: {zeros:,}, positives: {pos:,}, missing: {miss:,}")


Percent_Bleaching summary
count    34515.00
mean         9.62
std         20.19
min          0.00
25%          0.00
50%          0.25
75%          6.00
max        100.00
Name: Percent_Bleaching, dtype: float64
  zeros: 16,629, positives: 17,886, missing: 6,846


In [77]:
# categorizing the severity of bleaching, might not need this
df = df.dropna(subset=['Percent_Bleaching']).copy()
df['bleached'] = (df['Percent_Bleaching'] > 0).astype(int)
df['severity'] = pd.cut(
    df['Percent_Bleaching'],
    bins=[-1, 10, 50, 100],
    labels=['mild','moderate','severe']
)

In [80]:
print("\nFinal dataset:")
print(f"rows: {len(df):,}")
print(f"bleached counts:\n{df['bleached'].value_counts()}")
print(f"severity breakdown:\n{df['severity'].value_counts()}")


Final dataset:
rows: 34,515
bleached counts:
bleached
1    17886
0    16629
Name: count, dtype: int64
severity breakdown:
severity
mild        27579
moderate     4571
severe       2365
Name: count, dtype: int64


In [81]:
cleaned_path_csv = 'gcbd_cleaned.csv'

In [82]:
df.to_csv(cleaned_path_csv, index=False)
print(f"Saved cleaned data {cleaned_path_csv}")

Saved cleaned data → gcbd_cleaned.csv
